In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

@tf.custom_gradient
def grad_reverse(x, lambda_):
    def grad(dy):
        return -lambda_ * dy, None
    return x, grad

class GradientReversal(tf.keras.layers.Layer):
    def __init__(self, lambda_):
        super().__init__()
        self.lambda_ = lambda_

    def call(self, x):
        return grad_reverse(x, self.lambda_)

def build_feature_extractor(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3,3), activation='relu')(inputs)
    x = MaxPooling2D()(x)
    x = Conv2D(64, (3,3), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(inputs, x, name='feature_extractor')

def build_label_predictor():
    inputs = Input(shape=(128,))
    x = Dense(64, activation='relu')(inputs)
    outputs = Dense(10, activation='softmax')(x)
    return Model(inputs, outputs, name='label_predictor')

def build_domain_classifier(lambda_):
    inputs = Input(shape=(128,))
    x = GradientReversal(lambda_)(inputs)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)  
    return Model(inputs, outputs, name='domain_classifier')

input_shape = (28, 28, 3)  

feature_extractor = build_feature_extractor(input_shape)
label_predictor = build_label_predictor()
domain_classifier = build_domain_classifier(lambda_=1.0)

inputs = Input(shape=input_shape)
features = feature_extractor(inputs)
label_preds = label_predictor(features)
domain_preds = domain_classifier(features)

dann_model = Model(inputs=inputs, outputs=[label_preds, domain_preds])

dann_model.compile(optimizer=Adam(),
                   loss=['categorical_crossentropy', 'categorical_crossentropy'],
                   loss_weights=[1.0, 1.0],
                   metrics=['accuracy','accuracy'])

In [22]:
dann_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 28, 28, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_extractor   │ (None, 128)       │    224,320 │ input_layer_11[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ label_predictor     │ (None, 10)        │      8,906 │ feature_extracto… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ domain_classifier   │ (None, 2)         │      8,386 │ feature_extracto… │
│ (Functional)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 241,612 (943.80 KB)

 Trainable params: 241,612 (943.80 KB)

 Non-trainable params: 0 (0.00 B)

MNIST M

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

ds = load_dataset("Mike0307/MNIST-M")

import numpy as np

train_data = ds['train']
test_data = ds['test']

X_train_m = [np.array(image.resize((28, 28))) for image in train_data['image']]  # Resizing each image to (28, 28)
y_train_m = train_data['label']

X_test_m = [np.array(image.resize((28, 28))) for image in test_data['image']]  # Resizing each image to (28, 28)
y_test_m = test_data['label']

X_train_m = np.array(X_train_m)
X_test_m = np.array(X_test_m)
y_train_m = np.array(y_train_m)  
y_test_m = np.array(y_test_m)    

print('The current size of our dataset is : \nX_train -> ',X_train_m.shape,'\ny_train -> ' , y_train_m.shape, 
      '\nX_test -> ' , X_test_m.shape,'\ny_test' , y_test_m.shape )

The current size of our dataset is : 
X_train ->  (59001, 28, 28, 3) 
y_train ->  (59001,) 
X_test ->  (9001, 28, 28, 3) 
y_test (9001,)


In [24]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten

# Normalizing the image
'''
Converts pixel values from 0-255 (uint8) to 0.0-1.0 (float32).
This helps neural networks train faster and more accurately.
'''
X_train_m = X_train_m.astype(np.float32)/255
X_test_m = X_test_m.astype(np.float32)/255
print(len(X_train_m))

# we have converted our class to one hot vector 
y_train_m = keras.utils.to_categorical(y_train_m,10)
y_test_m = keras.utils.to_categorical(y_test_m,10)


59001


MNIST

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import keras 
from keras.datasets import mnist

(X_train,y_train),(X_test,y_test) = mnist.load_data()

print('The current size of our dataset is : \nX_train -> '
      ,X_train.shape,'\ny_train -> ' , y_train.shape, '\nX_test -> ' 
      , X_test.shape,'\ny_test' , y_test.shape )

The current size of our dataset is : 
X_train ->  (60000, 28, 28) 
y_train ->  (60000,) 
X_test ->  (10000, 28, 28) 
y_test (10000,)


In [26]:
# Normalizing the image
'''
Converts pixel values from 0-255 (uint8) to 0.0-1.0 (float32).
This helps neural networks train faster and more accurately.
'''
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255
print(len(X_train))

# expand the dimension to (28,28,1)
X_train = np.expand_dims(X_train,-1)
X_test = np.expand_dims(X_test,-1)

# we have converted our class to one hot vector 
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

X_train = np.repeat(X_train, 3, axis=-1)  # from (N,28,28,1) to (N,28,28,3)
X_test = np.repeat(X_test, 3, axis=-1)


60000


In [27]:

domain_source = np.tile([1,0], (len(X_train),1))  
domain_target = np.tile([0,1], (len(X_train_m),1))  

X_combined = np.concatenate([X_train, X_train_m], axis=0)
domain_combined = np.concatenate([domain_source, domain_target], axis=0)

y_combined = np.concatenate([y_train, np.zeros_like(y_train_m)], axis=0)

domain_labels = np.concatenate([
    np.zeros(len(X_train)), 
    np.ones(len(X_train_m))
])
domain_combined = to_categorical(domain_labels, 2)

In [28]:
batch_size = 128
epochs = 20

dann_model.fit(X_combined, [y_combined, domain_combined],
               batch_size=batch_size,
               epochs=epochs,
               validation_split=0.1)

Epoch 1/20
837/837 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - domain_classifier_accuracy: 0.4064 - domain_classifier_loss: 73222.3125 - label_predictor_accuracy: 0.1278 - label_predictor_loss: 2736347.2500 - loss: 2809500.0000 - val_domain_classifier_accuracy: 0.0000e+00 - val_domain_classifier_loss: 1847151.5000 - val_label_predictor_accuracy: 0.0000e+00 - val_label_predictor_loss: 0.0000e+00 - val_loss: 1847145.7500
Epoch 2/20
837/837 ━━━━━━━━━━━━━━━━━━━━ 22s 26ms/step - domain_classifier_accuracy: 0.5028 - domain_classifier_loss: 1331055.1250 - label_predictor_accuracy: 0.1071 - label_predictor_loss: 255007072.0000 - loss: 256337200.0000 - val_domain_classifier_accuracy: 0.0000e+00 - val_domain_classifier_loss: 362797.0312 - val_label_predictor_accuracy: 1.0000 - val_label_predictor_loss: 0.0000e+00 - val_loss: 362794.4062
Epoch 3/20
837/837 ━━━━━━━━━━━━━━━━━━━━ 24s 28ms/step - domain_classifier_accuracy: 0.4977 - domain_classifier_loss: 2190439.7500 - label_predictor_accuracy: 0.1050 - l

KeyboardInterrupt: 

In [ ]:


loss, label_loss, domain_loss, label_acc, domain_acc = dann_model.evaluate(X_test_m, [y_test_m, np.tile([0,1], (len(X_test_m),1))])

print(f"Target domain (MNIST-M) label accuracy: {label_acc:.4f}")

282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - domain_classifier_accuracy: 0.0000e+00 - domain_classifier_loss: 0.8271 - label_predictor_accuracy: 0.2134 - label_predictor_loss: 21388849053696.0000 - loss: 21388230393856.0000
Target domain (MNIST-M) label accuracy: 0.0000
